# Handling histological FOV data from BreastIHC API

Image and API responses are extracted from MindPeak's [BreastIHC](https://ihc.mindpeak.ai/) app (free registration required)

In [7]:
import pandas as pd
import json
from matplotlib import pyplot as plt
from matplotlib import figure
import numpy as np
import PIL
import math

%matplotlib inline

Responses and associated image from the Ki67 demo images - we can gather more data using uploaded breast cell image data (e.g. from [CAMELYON](http://gigadb.org/dataset/100439))

In [2]:
RESPONSES = [
    "labC_1.bd7b589b",
    "labB_1.fbc70936",
    "labB_0.0456cf3f",
    "labA_0.722f7db3"
]

RESPONSES_DIR = "Responses/"
IMAGES_DIR = "Images/"

CROP_WIDTH = 64

Parses the API response json and adds the associated image data 'thumbnails', both as np.array and a PIL, for each classified cell

In [12]:
def parse_api_response(responses, index):
    response_path = RESPONSES_DIR + "response_" + responses[index] + ".json"
    with open(response_path, "r") as f:
        data = json.load(f)
        cell_dict = data["response"]["results"]["cells"]
        df = pd.DataFrame.from_dict(cell_dict)
        
    image_path = IMAGES_DIR + responses[index] + ".png"
    img = plt.imread(image_path)
    
    add_cell_images_to_df(df, img,  CROP_WIDTH)
    return df

def add_cell_images_to_df(df, img, crop_width):
    image_data = []
    images = []
    for index, row in df.iterrows():
        center = (df.x[index] + 64, df.y[index] + 64) # center of the region
        crop_offset = math.floor(crop_width/2)
        cropped = img[(center[1] - crop_offset):(center[1] + crop_offset),
                   (center[0] - crop_offset):(center[0] + crop_offset),:]

        image_data.append(cropped)
        images.append(PIL.Image.fromarray(np.uint8(cropped*255)))
            
    df["image_data"] = image_data
    df["image"] = images

Reclassifies cell type based on 'intensity' data, I guess that this should be equivalent to moving the Positivity Threshold slider in the BreastIHC GUI, but the numbers don't seem to match up perfectly. Either there is some logic that I'm missing, or it's a bug!

In [9]:
def reclassify_by_intensity(df, intensity_threshold):
    df.loc[df.intensity < intensity_threshold, "type"] = "non_tumor"
    df.loc[df.intensity >= intensity_threshold, "type"] = "tumor"

Reproduces the BreastIHC GUI for a given api response. There is a 64 pixel margin around the FOV where there is no classification, assumedly to omit any classifications affected by border effects of the CNN

In [10]:
def display_api_response(responses, index, intensity_threshold):

    df = parse_api_response(RESPONSES, 0)
    reclassify_by_intensity(df, intensity_threshold)

    grouped_by_type = df.groupby("type")
    tumor_cells = grouped_by_type.get_group("tumor")
    non_tumor_cells = grouped_by_type.get_group("non_tumor")
    
    image_path = IMAGES_DIR + responses[index] + ".png"
    img = plt.imread(image_path)
    
    plt.imshow(img)
    point_size = 10
    plt.scatter(tumor_cells.x + 64, tumor_cells.y + 64, s = point_size, c = 'red')
    plt.scatter(non_tumor_cells.x + 64, non_tumor_cells.y + 64, s = point_size, c = 'green')

    fig = plt.gcf()
    fig.set_size_inches(18, 10 , forward=True)
    plt.show()
    print("tumor cells: " + str(len(tumor_cells.index))+ "\nnon-tumor cells: " + str(len(non_tumor_cells.index))) 

In [11]:
display_api_response(RESPONSES, index = 0, intensity_threshold = 0.5)

TypeError: slice indices must be integers or None or have an __index__ method

Display a number of random cell images from the parsed dataframe, along with its classification for some intensity threshold (to check that it's working!)

In [ ]:
intensity_threshold = 0.5
number_of_images_to_display = 1
cell_df = parse_api_response(RESPONSES, 0)
reclassify_by_intensity(cell_df, intensity_threshold)

for i in range(number_of_images_to_display):
    random_cell_index = np.random.randint(0, len(cell_df.index))

    plt.imshow(cell_df["image_data"][random_cell_index])
    plt.show()
    print(cell_df["type"][random_cell_index])